# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
# import json
import os

# Import API key
from config import key

In [2]:
# Load city data from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
city_data_df.dtypes

City_Name       object
City_Lat       float64
City_Long      float64
Temperature    float64
Humidity         int64
Cloudiness       int64
Wind_Speed     float64
Country         object
Date            object
dtype: object

In [3]:
# Rename columns and create new dataframe with dropped NA values 
city_data_df = city_data_df.rename(columns={'City_Name': 'City', 'City_Lat': 'Latitude', 
                                            'City_Long': 'Longitude', 'Wind_Speed': 'Wind Speed'})

city_loc = city_data_df.dropna(axis = 0, how ='any')
# city_data_cleandf = city_data_dropna[(city_data_dropna != 0).all(1)]

# comparing sizes of data frames
print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_loc), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_loc)))
city_loc.to_csv("./output_data/city_loc.csv")

Old data frame length: 645 
New data frame length: 639 
Number of rows with at least 1 NA value:  6


### Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key=key)

# Set Locations for the heatmap
locations = city_loc[["Latitude", "Longitude"]]

humidity = city_loc["Humidity"].astype(float)

In [5]:
# Plot humidity heatmap
fig = gmaps.figure()

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
# Add layer and display fig
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Ideal Weather Conditions

In [6]:
# Create new dataframe based on indicated parameters
ideal_cities = city_loc[(city_loc['Temperature']>70) & (city_loc['Temperature']<90) 
                   & (city_loc['Humidity']>40) & (city_loc['Humidity']<70) 
                   & (city_loc['Wind Speed']<10) & (city_loc['Cloudiness']<30)]
ideal_cities.head()
# len(ideal_cities)    #---- used to check number of cities that meet criteria; uncomment to enable 

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
13,Rome,34.2570,-85.1647,84.45,42,1,1.99,US,2021-06-17 17:57:41
14,Guarapari,-20.6667,-40.4975,73.89,68,14,9.42,BR,2021-06-17 17:59:43
32,Jablah,35.3614,35.9256,71.89,68,0,4.61,SY,2021-06-17 17:59:49
73,Broome,42.2506,-75.8330,70.92,41,1,1.99,US,2021-06-17 18:00:10
79,Hamilton,39.1834,-84.5333,83.10,44,19,1.99,US,2021-06-17 18:00:10


### Find Nearest Hotel

In [11]:
# Create new dataframe of city geo data; add new column to store hotel name
hotels_df = ideal_cities.loc[:, ['City', 'Country', 'Latitude', 'Longitude', 'Date']]
hotels_df['Hotel Name'] = ""
                             
# Parameters dictionary for Nearby search requests
params = {
    'location': 'city_geo',
    'types': 'lodging', # keyword
    'radius': 5000, 
    'key': key
}
# Nearby search base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through pandas dataframe
for index, row in ideal_cities.head().iterrows():
    city = row['City']
    lat = ideal_cities.loc[index]['Latitude']
    long = ideal_cities.loc[index]['Longitude']
    city_geo = (f'{lat},{long}')
    
    # Construct url and make API request
    print(f'Retrieving Results for Index {index}: {city}.')
    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        hotels_df.loc[index, 'Hotel Name'] = results[0]['Hotel Name']
    
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
    print('------------')

print(hotels_df)


Retrieving Results for Index 13: Rome.
Missing field/result... skipping.
------------
Retrieving Results for Index 14: Guarapari.
Missing field/result... skipping.
------------
Retrieving Results for Index 32: Jablah.
Missing field/result... skipping.
------------
Retrieving Results for Index 73: Broome.
Missing field/result... skipping.
------------
Retrieving Results for Index 79: Hamilton.
Missing field/result... skipping.
------------
                City Country  Latitude  Longitude                 Date  \
13              Rome      US   34.2570   -85.1647  2021-06-17 17:57:41   
14         Guarapari      BR  -20.6667   -40.4975  2021-06-17 17:59:43   
32            Jablah      SY   35.3614    35.9256  2021-06-17 17:59:49   
73            Broome      US   42.2506   -75.8330  2021-06-17 18:00:10   
79          Hamilton      US   39.1834   -84.5333  2021-06-17 18:00:10   
99           Russell      US   32.3502   -85.1999  2021-06-17 18:00:06   
222        Kununurra      AU  -15.7667 